In [1]:
import sys
sys.path.append("../..")

In [2]:
from src.marks_reader import *
from src.prediction_saver import *
from src.competition_data import *
from src.simulator import simulate_all_events
from src.ponderator import ponderate_all_events
from events_data import events, event_params
from datetime import datetime

In [3]:
entry_list = load_json('entry_list.json')

competition = CompetitionData(
    name="World Athletics Championships Oregon22", 
    events_data=events, 
    events_params=event_params,
    start_date=datetime(2022, 7, 15),
    entry_list=entry_list,
)

event_folder = 'classified'
only = None
top = 8
years_weight = {
    2022: 4,
    2021: 2,
    2020: 1,
}

In [4]:
data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

In [5]:
ponderated_data = ponderate_all_events(
    data=data, 
    competition=competition,
    years_weight=years_weight,   
)

In [6]:
prediction = simulate_all_events(
    data=ponderated_data, 
    competition=competition, 
    top=top, 
    times=1000, 
    models_folder='models',
    override_models=True,
    only=only,
    logs=True, 
)

Creating athletes models for event atl_100m and sex male...
(0.84 %) Creating kde model of ANT_cejhae_greene...
(1.68 %) Creating kde model of AUS_jake_doran...
(2.52 %) Creating kde model of AUS_joshua_azzopardi...
(3.36 %) Creating kde model of AUS_rohan_browning...
(4.20 %) Creating kde model of AUT_markus_fuchs...
(5.04 %) Creating kde model of BAH_samson_colebrooke...
(5.88 %) Creating kde model of BAH_terrence_jones...
(6.72 %) Creating kde model of BOT_letsile_tebogo...
(7.56 %) Creating kde model of BRA_erik_cardoso...
(8.40 %) Creating kde model of BRA_felipe_bardi...
(9.24 %) Creating kde model of BRA_rodrigo_do_nascimento...
(10.08 %) Creating kde model of CAN_aaron_brown...
(10.92 %) Creating kde model of CAN_andre_de_grasse...
(11.76 %) Creating kde model of CAN_bolade_ajomale...
(12.61 %) Creating kde model of CAN_jerome_blake...
(13.45 %) Creating kde model of CHN_bingtian_su...
(14.29 %) Creating kde model of CHN_zhiqiang_wu...
(15.13 %) Creating kde model of CIV_arthur

In [7]:
save_prediction(
    results=prediction, 
    competition=competition, 
    top=top,
)

In [5]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import joblib

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style

# Ajuste de distribuciones
# ==============================================================================
from sklearn.neighbors import KernelDensity

# Configuración matplotlib
# ==============================================================================
plt.rcParams['figure.figsize'] = (12,8)
style.use('ggplot') or plt.style.use('ggplot')


def density_pred(kde_model: KernelDensity, samples):    
    log_density_pred = kde_model.score_samples(X=samples.reshape(-1, 1))
    return np.exp(log_density_pred)

    
def plot_model(event, sex, model_name, bounds, Show=True):
    model: KernelDensity = joblib.load(f'models/{event}/{sex}/{model_name}.pkl')
    X = np.linspace(bounds[0], bounds[1], 200)
    Y = density_pred(model, X)
    X, Y = model.evaluate()
    plt.plot(X, Y, label=model_name)
    if Show:
        plt.show()

bounds = (1700, 1900)
event, sex = 'atl_jab', 'female'
plot_model(event, sex, 'AUS_alexandra_roberts', bounds, False)
# plot_model(event, sex, 'BRN_kalkidan_gezahegne', bounds, False)
# plot_model(event, sex, 'NED_sifan_hassan', bounds, False)
plt.legend()

plt.show()

model: KernelDensity = joblib.load(f'models/{event}/{sex}/NED_sifan_hassan.pkl')
print(model.bandwidth)

AttributeError: 'KernelDensity' object has no attribute 'evaluate'